<a href="https://colab.research.google.com/github/muhammed-saeed/Low-Resource-Cross-Lingual-Adaptive-Training-for-Nigerian-Pidgin/blob/main/T5_Translation_using_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers[sentencepiece] sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

# Fine-tuning a model on a translation task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a translation task. We will use the [WMT dataset](http://www.statmt.org/wmt16/), a machine translation dataset composed from a collection of various sources, including news commentaries and parliament proceedings.

![Widget inference on a translation task](images/translation.png)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [2]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"
model_checkpoint = "google/flan-t5-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`Helsinki-NLP/opus-mt-en-ro`](https://huggingface.co/Helsinki-NLP/opus-mt-en-ro) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`. We use the English/Romanian part of the WMT dataset here.

In [3]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/610320 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-3-97ced1cd8dd7>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 610320
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1999
    })
})

To access an actual element, you need to select a split first, then give an index:

In [5]:
raw_datasets["train"][0]
# https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/test.en



{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

In [6]:
!wget https://raw.githubusercontent.com/muhammed-saeed/Arabic-Distil-Bert/main/train.tsv
!wget https://raw.githubusercontent.com/muhammed-saeed/Arabic-Distil-Bert/main/eval.tsv

--2023-04-08 23:58:15--  https://raw.githubusercontent.com/muhammed-saeed/Arabic-Distil-Bert/main/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25254402 (24M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]  24.08M  --.-KB/s    in 0.1s    

2023-04-08 23:58:15 (166 MB/s) - ‘train.tsv’ saved [25254402/25254402]

--2023-04-08 23:58:16--  https://raw.githubusercontent.com/muhammed-saeed/Arabic-Distil-Bert/main/eval.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 740564 (723K) [text/

In [7]:
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/train.en
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/test.en
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/val.en
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/train.pcm
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/test.pcm
!wget https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/val.pcm


--2023-04-08 23:58:16--  https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/train.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5724862 (5.5M) [text/plain]
Saving to: ‘train.en’

train.en            100%[===================>]   5.46M  --.-KB/s    in 0.07s   

2023-04-08 23:58:16 (73.3 MB/s) - ‘train.en’ saved [5724862/5724862]

--2023-04-08 23:58:16--  https://raw.githubusercontent.com/muhammed-saeed/Naija-Pidgin/main/t5_translation/data/test.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

In [91]:
from datasets import Dataset, DatasetDict

# Load the sentences from the text files
english_sentences = []
pcm_sentences = []
data = []
with open("/content/train.en", "r", encoding="utf-8") as f:
    english_sentences = f.readlines()

with open("/content/train.pcm", "r", encoding="utf-8") as f:
    pcm_sentences = f.readlines()

# Align the sentences and create a list of dictionaries
data = [
    {"en": en.strip(), "pcm": pcm.strip()}
    for en, pcm in zip(english_sentences, pcm_sentences)
]

train_data = data

val_english_sentences = []
val_pcm_sentences = []
val_data = []

with open("/content/val.en", "r", encoding="utf-8") as f:
    val_english_sentences = f.readlines()

with open("/content/val.pcm", "r", encoding="utf-8") as f:
    val_pcm_sentences = f.readlines()





test_english_sentences = []
test_pcm_sentences = []
test_data = []
with open("/content/test.en", "r", encoding="utf-8") as f:
    test_english_sentences = f.readlines()

with open("/content/test.pcm", "r", encoding="utf-8") as f:
    test_pcm_sentences = f.readlines()

val_data = [
    {"en": en.strip(), "pcm": pcm.strip()}
    for en, pcm in zip(val_english_sentences, val_pcm_sentences)
]



# Align the sentences and create a list of dictionaries for test data
test_data = [
    {"en": en.strip(), "pcm": pcm.strip()}
    for en, pcm in zip(test_english_sentences, test_pcm_sentences)
]

# Create DatasetDict
our_datasets = DatasetDict(
    {
        "train": Dataset.from_dict({"translation": train_data}),
        "validation": Dataset.from_dict({"translation": val_data}),
        "test": Dataset.from_dict({"translation": test_data}),
    }
)

In [92]:
print(our_datasets["train"][0])

{'translation': {'en': 'Then he gave examples of being holy .', 'pcm': 'E come tell dem some things wey dem go dey do to show sey dem dey holy .'}}


In [95]:
our_datasets["validation"]['translation'][:10]




[{'en': 'Their words are foolish; they make agreements with false oaths, so punishment will come up like a poison-plant in a ploughed field.',
  'pcm': 'Na empty words dem dey tok kon dey give lie-lie promise and naw, just as dem turn field wey betta plant suppose grow to where poizin grass full, na so too dem don turn betta judgement to bad one.'},
 {'en': 'But he went to sleep again and had a second dream, in which he saw seven heads of grain, full and good, all on one stem.',
  'pcm': 'Den e sleep again kon dream anoda dream: E si seven korn head dey grow for one korn stik and dem dey fine and big well-well.'},
 {'en': 'Salt is good; but if the taste goes from it, how will you make it salt again? Have salt in yourselves, and be at peace one with another.And he got up, and went into the country of Judaea on the other side of Jordan: and great numbers of people came together to him again; and, as was his way, he gave them teaching.',
  'pcm': '“Salt good, but if e nor get taste again,

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [23]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'In committee, we decided to recommend increasing the future budget by 5%.', 'ro': 'Am decis în comisie să recomandăm creşterea bugetului viitor cu 5 %.'}"
1,"{'en': 'I lived more than 30 years of my life under dictatorship, those were the obstacles I ran into, it wasn't me who looked for those themes, they looked for me.""', 'ro': 'Am trăit peste 30 de ani din viaţă sub dictatură, acestea sunt piedicile pe care le-am întâlnit, nu am căutat eu aceste teme, ele m-au căutat pe mine”.'}"
2,"{'en': 'Many people wonder how it is possible to protect animals while they are being killed.', 'ro': 'Mulţi se întreabă cum este posibil să protejăm animalele în momentul uciderii acestora.'}"
3,"{'en': 'I would also like to thank you, Commissioner, for your positive cooperation, and also for doing something you understood to be important, namely, travelling to Northern Kosovo, even spending a night there to make it clear to the population that you are right behind them.', 'ro': 'Aş dori să vă mulţumesc, domnule comisar, pentru cooperarea dumneavoastră pozitivă şi pentru că aţi făcut ceva ce aţi înţeles că este important, şi anume călătoria în nordul Kosovo, rămânând chiar şi o noapte acolo pentru ca populaţia să înţeleagă în mod clar că îi sprijiniţi îndeaproape.'}"
4,"{'en': 'This would have serious consequences for the general market, and that is why this issue should be given serious thought.', 'ro': 'Acest lucru ar avea consecinţe grave pentru piaţă în general, motiv pentru care trebuie să se reflecteze serios asupra acestei chestiuni.'}"


In [12]:
show_random_elements(our_datasets["train"])

,translation
0,"{'en': 'Kiriath-baal (which is Kiriath- jearim), and Rabbah; two towns with their unwalled places.', 'pcm': 'Kiriat-Baal (wey bi Kiriat-Jearim) and Rabbah and dem bi two towns.'}"
1,"{'en': 'And rain came down on the earth for forty days and forty nights.', 'pcm': 'den rain from heaven kon start to fall for di eart for forty days and forty nites.'}"
2,"{'en': 'Then keep the feast of weeks to the Lord your God, with an offering freely given to him from the wealth he has given you:', 'pcm': 'Den make una celebrate Harvest Feast to honor di Oga una God. Make una gi-am ofrin from una mind, akordin to how E bless una rish.'}"
3,"{'en': 'And when there had been much discussion, Peter got up and said to them, My brothers, you have knowledge that some time back it was God's pleasure that by my mouth the good news might be given to the Gentiles so that they might have faith.', 'pcm': 'Afta dem don argue well-well about di matter, Pita kon stand up sey, “My brodas, una know sey before- before, na mi God choose make e go prish give Gentile pipol make dem hear and bilive en word.'}"
4,"{'en': 'But they went against him, causing grief to his holy spirit: so he was turned against them, and made war on them.', 'pcm': 'but dem fight against am kon make am vex. So God kon bi dia enemy and E fight against dem too.'}"


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [13]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [14]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

For the mBART tokenizer (like we have here), we need to set the source and target languages (so the texts are preprocessed properly). You can check the language codes [here](https://huggingface.co/facebook/mbart-large-cc25) if you are using this notebook on a different pairs of languages.

In [16]:
if "mbart" in model_checkpoint:
    print("hte model is mBART")
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ro-RO"

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [17]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [30]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [18]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


If you are using one of the five T5 checkpoints that require a special prefix to put before the inputs, you should adapt the following cell.

In [19]:
if model_checkpoint in ["google/flan-t5-base","t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
    print("The model is T5")
else:
    prefix = ""

The model is T5


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [36]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [65]:
max_input_length = 128
max_target_length = 128
our_source_lang = "en"
our_target_lang = "pcm"

def our_preprocess_function(examples):
    inputs = []
    targets = []
    for ex in examples["translation"]:
        try:
            inputs.append(prefix + ex[our_source_lang])
            targets.append(ex[our_target_lang])
        except KeyError as e:
            print(f"Error in example: {ex}")
            print(f"KeyError: {e}")

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs




In [107]:
def our_preprocess_function_bidirectional(examples):
    inputs = []
    targets = []
    reverse_inputs = []
    reverse_targets = []
    
    for ex in examples["translation"]:
        try:
            inputs.append("translate english to pcm: " + ex[our_source_lang])
            targets.append(ex[our_target_lang])
            reverse_inputs.append("translate pcm to english: " + ex[our_target_lang])
            reverse_targets.append(ex[our_source_lang])
        except KeyError as e:
            print(f"Error in example: {ex}")
            print(f"KeyError: {e}")

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    reverse_model_inputs = tokenizer(reverse_inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
        reverse_labels = tokenizer(reverse_targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    reverse_model_inputs["labels"] = reverse_labels["input_ids"]
    
    # Combine both directions
    for key in model_inputs.keys():
        model_inputs[key].extend(reverse_model_inputs[key])
    
    return model_inputs


This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [21]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[13959, 1566, 12, 3871, 29, 10, 19428, 13, 12876, 10, 217, 13687, 7, 1], [13959, 1566, 12, 3871, 29, 10, 2276, 8843, 138, 13, 13687, 7, 13, 1767, 3823, 10, 217, 13687, 7, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[4961, 106, 20437, 13636, 252, 10, 3, 9, 142, 11002, 15948, 18, 11868, 138, 1], [5085, 5840, 498, 6345, 252, 18, 11868, 138, 491, 28109, 15, 23, 17875, 15, 10, 3, 9, 142, 11002, 15948, 18, 11868, 138, 1]]}

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [38]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)


In [32]:
raw_datasets["train"][10]

{'translation': {'en': 'Membership of Parliament: see Minutes',
  'ro': 'Componenţa Parlamentului: a se vedea procesul-verbal'}}

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

In [96]:
our_tokenized_datasets = our_datasets.map(our_preprocess_function, batched=True)

Map:   0%|          | 0/47755 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/2103 [00:00<?, ? examples/s]

In [108]:
our_tokenized_datasets_bidirectional = our_datasets.map(our_preprocess_function_bidirectional, batched=True)

ArrowInvalid: Column 1 named input_ids expected length 1000 but got length 2000

Map:   0%|          | 0/47755 [00:00<?, ? examples/s]

ArrowInvalid: ignored

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [97]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [102]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/marian-finetuned-en-to-ro"` or `"huggingface/marian-finetuned-en-to-ro"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [99]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [100]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [103]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)




We can now finetune our model by just calling the `train` method:

In [104]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


KeyboardInterrupt: ignored

You can now upload the result of the training to the Hub, just execute this instruction:

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```